# RENergetic (OSR predictive tests on Use Case 1)
## Algorithm LSTM 
.
.
.
#### ALGORITHM TYPE: Deep Learning LSTM Neural Network

#### OBJECTIVE: prediction of heat energy consumption (MWh) in Dibit 2 building in OSR

#### Samplig Rate: 1 hr 

#### PREDICTION: prediction is single step (a single hour forward prediction)

.
.
.


#### MW consumption Dibit2 from 8-Nov-2020 07:00:00 to 9-Nov-2020 22:00:00

#### PREDICT NEXT HOUR ---> 9-Nov-2020 23:00:00

###### NOTE @21-10-21 model is for demo purpose only (no cross-validation and hypermparmenter optimization is provided)


In [1]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense               # note!: let's keep hidden layers easy: 1 dense ok!

Using TensorFlow backend.
C:\Users\DANIELE\Anaconda3\envs\test_Models_2021\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\DANIELE\Anaconda3\envs\test_Models_2021\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\DANIELE\Anaconda3\envs\test_Models_2021\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users

In [2]:
def split_sequence(sequence, n_steps):                                  # split sequence in 2D
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        
        if end_ix > len(sequence)-1:                                     # check break
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
        return array(X), array(y)

In [3]:
 # let's enter a 10x4 (sequenceXtimesteps) 
                                        # MW consumption Dibit2 from 8-Nov-2020 07:00:00 to 9-Nov-2020 22:00:00
                                        # LET'S PREDICT NEXT HOUR ---> 9-Nov-2020 23:00:00 ??
                                             
MW_sequence = [2.330,2.260,2.320,2.160,
               2.100,1.750,1.670,1.710,
               1.600,1.700,1.680,1.700,
               1.980,1.850,1.940,1.980,
               2.060,2.160,2.280,2.190,
               2.250,2.250,2.250,2.260,
               2.300,2.260,2.060,2.190,
               2.050,1.680,1.890,1.550,
               1.880,1.760,1.730,1.740,
               1.760,1.760,1.910,2.010]  # let's make temporal forwar prediction for next hour MW consumption 

In [4]:
n_steps = 4
n_features = 1
X, y = split_sequence(MW_sequence, n_steps)                               # split into samples
X = X.reshape((X.shape[0], X.shape[1], n_features))                       # reshape to 3D[samples, timesteps, features]

In [5]:
model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(n_steps, n_features))) #LSTM model!!!
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=200, verbose=0)                                    # fit method

x_input = array([1.760,1.760,1.910,2.010])                              # take last 4-samples to make prediction
# NEW_input = array([1.880,1.760,1.730,1.740])                          # take preceeding 4 samples to make prediction 

x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)                                  # predict method

### forecast value 
###### next hour (x_input) is expected MW 2.170   ;     
###### next hour (NEW_input) is expected MW 1.760

In [6]:
print('next hour prediction: ', yhat)

next hour prediction:  [[1.5617996]]


## LSTM model stored in RENergetic platform 
### (option: use a Docker container to operate on this python .py file)

In [7]:
import pickle5 as pickle
# save the model to disk
filename = 'OSR_example_RENERGETIC_LSTM_model.sav'
pickle.dump(model, open(filename, 'wb'))

## some time later...

## ...deploy LSTM model on new incoming data stream 
#### (a 4 hours stream input to forecast the fifth hour) 

In [8]:
NEW_input = array([1.880,1.760,1.730,1.740])                #read commented NEW_input above...so expected 1hr forward is MW 1.760 
NEW_input = NEW_input.reshape((1, n_steps, n_features))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))            #note:filename is 'OSR_example_RENERGETIC_LSTM_model.sav'
result = loaded_model.predict(NEW_input, verbose=0)
print('next hour prediction: ', result)

next hour prediction:  [[1.5058784]]


end